In [37]:
import pandas as pd
import numpy as np

# Sample DataFrame
data = [1.73,0,0,15.87,0,0, 24.34]
df = pd.DataFrame(data, columns=['values'])
df


,values
0,1.73
1,0.00
2,0.00
3,15.87
4,0.00
5,0.00
6,24.34


In [40]:
df.ewm(span=3, adjust=False).mean()

,values
0,1.730000
1,0.865000
2,0.432500
3,8.151250
4,4.075625
5,2.037812
6,13.188906
